# Saved States Overview

In [5]:
from core import game
save_dir = "saves/ai_battles/ai_battle_001"

In [6]:
import os
pkl_files = []
for file in os.listdir(save_dir):
    if file.endswith(".pkl"):
        pkl_files.append(file)

print("Found pkl files:", pkl_files)

Found pkl files: ['turn_025.pkl', 'turn_010.pkl', 'turn_005.pkl', 'turn_013.pkl', 'turn_024.pkl', 'turn_017.pkl', 'turn_026.pkl', 'turn_000.pkl', 'latest.pkl', 'turn_002.pkl', 'turn_001.pkl', 'turn_009.pkl', 'turn_012.pkl', 'turn_020.pkl', 'turn_019.pkl', 'turn_006.pkl', 'turn_007.pkl', 'turn_028.pkl', 'turn_015.pkl', 'turn_014.pkl', 'turn_022.pkl', 'turn_023.pkl', 'turn_018.pkl', 'turn_008.pkl', 'turn_021.pkl', 'turn_016.pkl', 'turn_027.pkl', 'turn_004.pkl', 'turn_029.pkl', 'turn_003.pkl', 'turn_011.pkl']


In [7]:
import pickle
import json
# select_files = pkl_files.copy()
select_files = ["latest.pkl"]
for file in select_files:
    with open(os.path.join(save_dir, file), "rb") as f:
        state = pickle.load(f)
        print(f"=== Loaded state from {file}: ===")
        print(json.dumps(state, indent=2))

=== Loaded state from latest.pkl: ===
{
  "state": {
    "players": [
      {
        "name": "AI Player 1",
        "roster": [
          "Pufferfish",
          "Sunfish",
          "Sea Wolf",
          "Sea Turtle",
          "Octopus",
          "Hammerhead Shark",
          "Clownfish",
          "Mimic Fish"
        ],
        "team": {
          "fish": [
            {
              "name": "Archerfish",
              "hp": -21,
              "atk": 100,
              "revealed": false,
              "buffs": [],
              "shields": 0,
              "dodge_chance": 0.0,
              "used_active_count": 0,
              "mimic_source": null
            },
            {
              "name": "Electric Eel",
              "hp": -23,
              "atk": 100,
              "revealed": false,
              "buffs": [],
              "shields": 0,
              "dodge_chance": 0.0,
              "used_active_count": 0,
              "mimic_source": null
            },
        

# Inference testing

In [38]:
from langchain_ollama import ChatOllama
model = "gpt-oss:20b"
temperature = 0.1
top_p = 0.9

llm = ChatOllama(
    model=model,
    temperature=temperature,
    top_p=top_p,
)
response = llm.invoke("What is the capital of France?")
print("LLM response:", response)

LLM response: content='The capital of France is **Paris**.' additional_kwargs={} response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-08-15T18:18:15.029567396Z', 'done': True, 'done_reason': 'stop', 'total_duration': 90705936382, 'load_duration': 89579361336, 'prompt_eval_count': 74, 'prompt_eval_duration': 577526484, 'eval_count': 40, 'eval_duration': 548376871, 'model_name': 'gpt-oss:20b'} id='run--f33e3fa7-0680-4afc-9776-3f3a5d116d0e-0' usage_metadata={'input_tokens': 74, 'output_tokens': 40, 'total_tokens': 114}


In [40]:
import json
res_json = json.loads(response.model_dump_json())
print(json.dumps(res_json, indent=2))

{
  "content": "The capital of France is **Paris**.",
  "additional_kwargs": {},
  "response_metadata": {
    "model": "gpt-oss:20b",
    "created_at": "2025-08-15T18:18:15.029567396Z",
    "done": true,
    "done_reason": "stop",
    "total_duration": 90705936382,
    "load_duration": 89579361336,
    "prompt_eval_count": 74,
    "prompt_eval_duration": 577526484,
    "eval_count": 40,
    "eval_duration": 548376871,
    "model_name": "gpt-oss:20b"
  },
  "type": "ai",
  "name": null,
  "id": "run--f33e3fa7-0680-4afc-9776-3f3a5d116d0e-0",
  "example": false,
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 74,
    "output_tokens": 40,
    "total_tokens": 114
  }
}
